# Data Cleaning Project: Energy Bills

## Mission

.Your goal is to transform the file received by management into a usable file.

For the accounting teams, "usable" means they should be able to search for the name of a specific branch and immediately get its energy consumption history to check for any anomalies.

To make this history useful, they need the average daily consumption. This way, they can compare, for example, a February with 28 days to a January with 31 days, without thinking that the decrease is just due to the difference in the number of days in the month.

In [1]:
# Import the libraries and load the file.
import pandas as pd
import numpy as np

energy = pd.read_csv("energy.csv", delimiter="|")

# Display the first few rows.
energy.head()

,SITE_NAME,INVOICE_NAME,BEGIN,END,KWH_PERIOD,PERIOD_EUR
0,12345_GEO (0001BB),1300082549,01/01/2018,31/01/2018,2 447,"316,46"
1,12345_GEO (0001BB),1300082549,01/02/2018,28/02/2018,2 211,"286,82"
2,12345_GEO (0001BB),1300082549,01/03/2018,31/03/2018,2 450,"317,32"
3,12345_GEO (0001BB),1300082549,01/04/2018,30/04/2018,2 431,"315,00"
4,12345_GEO (0001BB),1300082549,01/05/2018,31/05/2018,2 574,"333,37"


## The tasks:

#### 1) The name of a branch follows a specific format: it always consists of 4 digits and 2 letters. You are asked to remove any other characters from the "SITE_NAME" column.

In [2]:
# Extract SITE_NAME using regex pattern
energy['SITE_NAME'] = energy['SITE_NAME'].str.extract(r'(\d{4}[A-Za-z]{2})')

# Check for NaN and inf values in the dataframe and drop them
print(f"Missing values before cleaning : {energy['SITE_NAME'].isnull().sum()}")
mask = energy.isin([np.nan, np.inf, -np.inf]).any(axis=1)
energy = energy[~mask]
print(f"Missing values after cleaning : {energy['SITE_NAME'].isnull().sum()}")

energy.head()

Missing values before cleaning : 11
Missing values after cleaning : 0


,SITE_NAME,INVOICE_NAME,BEGIN,END,KWH_PERIOD,PERIOD_EUR
0,0001BB,1300082549,01/01/2018,31/01/2018,2 447,"316,46"
1,0001BB,1300082549,01/02/2018,28/02/2018,2 211,"286,82"
2,0001BB,1300082549,01/03/2018,31/03/2018,2 450,"317,32"
3,0001BB,1300082549,01/04/2018,30/04/2018,2 431,"315,00"
4,0001BB,1300082549,01/05/2018,31/05/2018,2 574,"333,37"


#### 2)You are asked to delete the "INVOICE_NAME" column, as it is not useful for the teams.

In [3]:
# Delete the column "INVOICE_NAME"
energy.drop('INVOICE_NAME', axis=1, inplace=True)

#### 3) Some branches receive monthly invoices, while others receive them less frequently (quarterly, semi-annually, annually). Add a column to your table that calculates the number of days between the start and end of the billing period

In [4]:
# Convert the columns 'BEGIN' and 'END' in date format
energy['BEGIN'] = pd.to_datetime(energy['BEGIN'], format='%d/%m/%Y')
energy['END'] = pd.to_datetime(energy['END'], format='%d/%m/%Y')

# Calculate the number of days (+ 1 to include both dates)
energy['NB_DAYS'] = (energy['END'] - energy['BEGIN']).dt.days +1

energy.head()

,SITE_NAME,BEGIN,END,KWH_PERIOD,PERIOD_EUR,NB_DAYS
0,0001BB,2018-01-01,2018-01-31,2 447,"316,46",31
1,0001BB,2018-02-01,2018-02-28,2 211,"286,82",28
2,0001BB,2018-03-01,2018-03-31,2 450,"317,32",31
3,0001BB,2018-04-01,2018-04-30,2 431,"315,00",30
4,0001BB,2018-05-01,2018-05-31,2 574,"333,37",31


#### 4) Based on this new column, create another column that provides the average daily consumption for the period.

In [5]:
# Convert the column 'KWH_PERIOD' to numeric without the space
energy['KWH_PERIOD'] = pd.to_numeric(energy['KWH_PERIOD'].str.replace(' ', ''))

# Create a new column and store the calculated the daily consumption
energy['KWH_DAY'] = energy['KWH_PERIOD'] / energy['NB_DAYS']

energy.head()

,SITE_NAME,BEGIN,END,KWH_PERIOD,PERIOD_EUR,NB_DAYS,KWH_DAY
0,0001BB,2018-01-01,2018-01-31,2447,"316,46",31,78.935484
1,0001BB,2018-02-01,2018-02-28,2211,"286,82",28,78.964286
2,0001BB,2018-03-01,2018-03-31,2450,"317,32",31,79.032258
3,0001BB,2018-04-01,2018-04-30,2431,"315,00",30,81.033333
4,0001BB,2018-05-01,2018-05-31,2574,"333,37",31,83.032258


#### 5) Also, please add a column that shows the total number of invoices available for each branch. It's okay if this information is repeated on each line for the same branch.

In [6]:
# Add a new column with the number of bills for each site
energy['NB_BILLS'] = energy.groupby('SITE_NAME')['SITE_NAME'].transform('count')

energy.head()

,SITE_NAME,BEGIN,END,KWH_PERIOD,PERIOD_EUR,NB_DAYS,KWH_DAY,NB_BILLS
0,0001BB,2018-01-01,2018-01-31,2447,"316,46",31,78.935484,35
1,0001BB,2018-02-01,2018-02-28,2211,"286,82",28,78.964286,35
2,0001BB,2018-03-01,2018-03-31,2450,"317,32",31,79.032258,35
3,0001BB,2018-04-01,2018-04-30,2431,"315,00",30,81.033333,35
4,0001BB,2018-05-01,2018-05-31,2574,"333,37",31,83.032258,35


#### 6)And as a bonus, if you can, the teams would greatly appreciate a list of all the branches, including the coefficient of variation of consumption for each branch. This will provide insight into the variability of energy consumption for each branch.

In [7]:
# Filter out rows with NaN or zero values
energy_filtered = energy[energy['KWH_DAY'].notna() & (energy['KWH_DAY'] != 0)]

# Calculate the coefficient of variation for each site
cv = energy_filtered.groupby('SITE_NAME')['KWH_DAY'].apply(lambda x: np.std(x) / np.mean(x)).reset_index()

# Rename the columns
cv.columns = ['SITE_NAME', 'CV_CONSUMPTION']

# Sort by descending coefficient of variation
cv = cv.sort_values('CV_CONSUMPTION', ascending=False)

# Print the resulting dataframe
cv.head()

,SITE_NAME,CV_CONSUMPTION
530,0195KO,2.941510
1454,0621KO,2.708906
318,0116BB,0.974172
1722,1081BB,0.904255
1419,0593BB,0.842247


#### 7) Finally, you need to export these two resulting files so you can send them to the operator's team and ask if this is what they expected from you

In [8]:
# Export the file calculating the coefficient of variation
cv.to_csv('cv_consumption.csv', index=False)

# Export the energy dataframe with modifications
energy.to_csv('energy_cleaned.csv', index=False)
